In [ ]:
import requests
from typing import List
import xml.etree.ElementTree as ET
import xml.dom.minidom
from google.colab import userdata
import os
import re

In [ ]:

url_search = "https://www.law.go.kr/DRF/lawSearch.do"
url_service = "https://www.law.go.kr/DRF/lawService.do"
params = {
    "OC": userdata.get('api_key'),
    "target": "law",
    "type": "XML",
    "query": "법인세법"

}

response1 = requests.get(url_search, params = params)
response1.raise_for_status()
root = ET.fromstring(response1.text)


In [ ]:
xml_str = ET.tostring(root, encoding="utf-8")
pretty_xml = xml.dom.minidom.parseString(xml_str).toprettyxml(indent="  ")
print(pretty_xml)

<?xml version="1.0" ?>
<LawSearch>
  <target>law</target>
  <키워드>법인세법</키워드>
  <section>lawNm</section>
  <totalCnt>3</totalCnt>
  <page>1</page>
  <numOfRows>3</numOfRows>
  <resultCode>00</resultCode>
  <resultMsg>success</resultMsg>
  <law id="1">
    <법령일련번호>276111</법령일련번호>
    <현행연혁코드>현행</현행연혁코드>
    <법령명한글>법인세법</법령명한글>
    <법령약칭명/>
    <법령ID>001563</법령ID>
    <공포일자>20251001</공포일자>
    <공포번호>21065</공포번호>
    <제개정구분명>타법개정</제개정구분명>
    <소관부처코드>1051000</소관부처코드>
    <소관부처명>기획재정부</소관부처명>
    <법령구분명>법률</법령구분명>
    <공동부령정보/>
    <시행일자>20260102</시행일자>
    <자법타법여부/>
    <법령상세링크>/DRF/lawService.do?OC=darlene1209&amp;target=law&amp;MST=276111&amp;type=HTML&amp;mobileYn=&amp;efYd=20260102</법령상세링크>
  </law>
  <law id="2">
    <법령일련번호>280853</법령일련번호>
    <현행연혁코드>현행</현행연혁코드>
    <법령명한글>법인세법 시행령</법령명한글>
    <법령약칭명/>
    <법령ID>003608</법령ID>
    <공포일자>20251230</공포일자>
    <공포번호>35947</공포번호>
    <제개정구분명>타법개정</제개정구분명>
    <소관부처코드>1051000</소관부처코드>
    <소관부처명>기획재정부</소관부처명>
    <법령구분명>대통령령</법령구분명>
    <공동

In [ ]:
def law_info(root, law_name):
  for law in root.findall(".//law"):
    name = law.find("법령명한글").text
    mst = law.find("법령일련번호").text
    date = law.find("시행일자").text

    if name == law_name:
      return name, mst, date

In [ ]:
law_info(root, "법인세법")




('법인세법', '276111', '20260102')

In [ ]:
def get_law_content(name, mst):
  params_service = {
      "OC": userdata.get('api_key'),
      "target": "law",
      "type": "XML",
      "MST": mst,
  }

  response2 = requests.get(url_service, params = params_service)
  response2.raise_for_status()
  return response2.text

In [ ]:
content_법인세 = get_law_content('법인세법', '276111')
root2 = ET.fromstring(content_법인세)

xml_str2 = ET.tostring(root2, encoding="utf-8")
pretty_xml2 = xml.dom.minidom.parseString(xml_str2).toprettyxml(indent="  ")
print(pretty_xml2[:5000])

<?xml version="1.0" ?>
<법령 법령키="0015632025100121065">
  

  <기본정보>
    

    <법령ID>001563</법령ID>
    

    <공포일자>20251001</공포일자>
    

    <공포번호>21065</공포번호>
    

    <언어>한글</언어>
    

    <법종구분 법종구분코드="A0002">법률</법종구분>
    

    <법령명_한글>법인세법</법령명_한글>
    

    <법령명_한자>법인세법</법령명_한자>
    

    <법령명약칭/>
    

    <제명변경여부>N</제명변경여부>
    

    <한글법령여부>N</한글법령여부>
    

    <편장절관>20050000</편장절관>
    

    <소관부처 소관부처코드="1051000">기획재정부</소관부처>
    

    <전화번호>044-215-4221</전화번호>
    

    <시행일자>20251001</시행일자>
    

    <제개정구분>타법개정</제개정구분>
    

    <조문시행일자문자열>20260102:제23조제2항제2호, 제24조, 제98조의5제1항, 제112조의2제3항</조문시행일자문자열>
    

    <별표편집여부>N</별표편집여부>
    

    <공포법령여부>Y</공포법령여부>
    

    <연락부서>
      <부서단위 부서키="398621">
        

        <소관부처명>기획재정부</소관부처명>
        

        <소관부처코드>1051000</소관부처코드>
        

        <부서명>법인세제과</부서명>
        

        <부서연락처>044-215-4221</부서연락처>
        

      </부서단위>
      <부서단위 부서키="398623">
        

        <소관부처명>기획재정부</소관부처명>
        

        <소관부처코드>1

In [ ]:
class load_law():
  def __init__(self, law_name):
    self.law_name = law_name
    self.law_dict = {} # {법령명: {mst, 시행일자}}

  def get_list(self):
    url_search = "https://www.law.go.kr/DRF/lawSearch.do"
    params = {
        "OC": userdata.get('api_key'),
        "target": "law",
        "type": "XML",
        "query": self.law_name
    }

    response1 = requests.get(url_search, params = params)
    response1.raise_for_status()
    root = ET.fromstring(response1.text)

    for law in root.findall(".//law"):
      name = law.find("법령명한글").text
      mst = law.find("법령일련번호").text
      date = law.find("시행일자").text

      self.law_dict[name] = {"mst": mst, "date": date}

    return self.law_dict

  @staticmethod
  def change_name(name):
    name = name.strip().replace(" ", "_")
    name = re.sub(r'[///:*?"<>|]+', '_', name)
    return name

  def get_content(self):
    url_service = "https://www.law.go.kr/DRF/lawService.do"
    base_dir = '/content/drive/MyDrive/rag/load_data'

    for name, info in self.law_dict.items():
      mst = info['mst']
      date = info['date']

      params_service = {
          "OC": userdata.get('api_key'),
          "target": "law",
          "type": "XML",
          "MST": mst,
      }

      response2 = requests.get(url_service, params = params_service)
      response2.raise_for_status()

      name = self.change_name(name)

      with open(f'{base_dir}/{name}.xml', 'w', encoding="utf-8") as f:
        f.write(response2.text)

    return self.law_dict


In [ ]:
법인세법 = load_law('법인세법')
법인세법.get_list()

{'법인세법': {'mst': '276111', 'date': '20260102'},
 '법인세법 시행령': {'mst': '280853', 'date': '20260102'},
 '법인세법 시행규칙': {'mst': '282643', 'date': '20260102'}}

In [ ]:
법인세법.get_content()

{'법인세법': {'mst': '276111', 'date': '20260102'},
 '법인세법 시행령': {'mst': '280853', 'date': '20260102'},
 '법인세법 시행규칙': {'mst': '282643', 'date': '20260102'}}

In [ ]:
cd './drive/MyDrive/rag/load_data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
소득세법 = load_law('소득세법')
소득세법.get_list()

{'소득세법': {'mst': '276127', 'date': '20260102'},
 '소득세법 시행령': {'mst': '280865', 'date': '20260102'},
 '소득세법 시행규칙': {'mst': '282651', 'date': '20260102'}}

In [ ]:
소득세법.get_content()

{'소득세법': {'mst': '276127', 'date': '20260102'},
 '소득세법 시행령': {'mst': '280865', 'date': '20260102'},
 '소득세법 시행규칙': {'mst': '282651', 'date': '20260102'}}

In [ ]:
부가가치세법 = load_law('부가가치세법')
부가가치세법.get_list()

{'부가가치세법': {'mst': '276117', 'date': '20260102'},
 '부가가치세법 시행령': {'mst': '280855', 'date': '20260102'},
 '부가가치세법 시행규칙': {'mst': '282645', 'date': '20260102'}}

In [ ]:
부가가치세법.get_content()

{'부가가치세법': {'mst': '276117', 'date': '20260102'},
 '부가가치세법 시행령': {'mst': '280855', 'date': '20260102'},
 '부가가치세법 시행규칙': {'mst': '282645', 'date': '20260102'}}